In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from parse import read_excel
from desdeo_tools.utilities import fast_non_dominated_sort_indices
import plotly.express as px

In [79]:
df = read_excel()
df

,CATID,BREEDGROUP,SEX,AGE_BEHAVIOUR,OTHER_CATS,problematic_behavior,seeks_physical_contact_people,Opens_doors,Rests _in_secure_places,Reacts_to_video_TV,...,restlessness_household_member_leave,exited_other_pets,Human_oriented,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,1,Siberian_and_Neva_Masquerade,Female,5.079872,unknown,unknown,5,5.0,3.0,5.0,...,4.0,NaN,5.0,-0.592536,-0.376839,0.677554,0.247061,1.318160,-0.370580,-0.329714
1,2,Sphynx_and_Devon_Rex,Female,4.225506,unknown,unknown,5,5.0,5.0,2.0,...,1.0,NaN,4.0,0.961022,1.890786,-0.585893,-1.319077,-1.185782,-0.758987,0.986191
2,3,Oriental,Female,7.419576,NO,A_little,5,5.0,4.0,4.0,...,5.0,NaN,NaN,-0.480038,-0.288479,0.583600,0.183466,1.313598,-0.127347,-0.009533
3,4,Sphynx_and_Devon_Rex,Female,2,YES,No,5,4.0,4.0,5.0,...,1.0,NaN,3.0,-0.876063,-0.696218,1.255442,0.899894,0.475639,-0.514913,-0.259347
4,5,House_cat,Female,3.535049,YES,A_little,5,1.0,4.0,1.0,...,1.0,NaN,3.0,-0.746316,-0.619940,-0.578864,-0.337497,-0.333070,3.084435,-0.113739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4311,4401,Korat,Female,0.333333,unknown,unknown,5,4.0,2.0,5.0,...,1.0,NaN,3.0,-1.196693,-0.315187,1.305526,0.822008,0.010866,-0.650696,-0.371469
4312,4402,Korat,Female,0.333333,unknown,unknown,5,4.0,3.0,5.0,...,1.0,NaN,3.0,-1.563574,0.855251,1.400834,0.559283,-0.376932,-0.630812,-0.374776
4313,4403,Korat,Female,0.333333,unknown,unknown,5,4.0,3.0,5.0,...,1.0,NaN,3.0,-1.515542,0.745532,1.412664,0.521732,-0.075399,-0.593704,-0.453170
4314,4404,Korat,Male,0.333333,unknown,unknown,5,4.0,3.0,5.0,...,1.0,NaN,3.0,-1.218929,-0.759734,1.034383,0.856490,0.106618,-0.746913,-0.483224


In [80]:
objectives = ["fearfulness", "human_aggression", "activity_playfulness", "cat_sociability", "human_sociability", "litterbox_issues", "excessive_grooming"]
directions = [1, 1, -1, -1, -1, 1, 1]

df[["activity_playfulness", "cat_sociability", "human_sociability"]] = -df[["activity_playfulness", "cat_sociability", "human_sociability"]]
data = df[objectives].to_numpy()

nadir, ideal = np.max(data, axis=0), np.min(data, axis=0)

data[0:10]

array([[-0.59253585, -0.37683853, -0.6775545 , -0.24706059, -1.31815959,
        -0.37057989, -0.32971416],
       [ 0.96102204,  1.89078581,  0.58589255,  1.31907745,  1.18578233,
        -0.75898746,  0.98619081],
       [-0.48003798, -0.28847854, -0.5835999 , -0.1834663 , -1.31359776,
        -0.12734664, -0.00953309],
       [-0.87606295, -0.69621826, -1.25544175, -0.89989358, -0.47563902,
        -0.51491316, -0.2593467 ],
       [-0.74631622, -0.61993966,  0.57886396,  0.33749746,  0.3330697 ,
         3.08443514, -0.11373859],
       [ 0.37926201, -0.81264175, -0.34348263, -0.72272709,  0.50268407,
        -1.14200693, -0.65903959],
       [ 2.83112696,  1.45468584,  1.13414467, -0.95598809,  1.91753782,
         3.56477328,  2.18691956],
       [-1.00821871, -0.67343642, -1.42147952, -0.34164396, -0.85067539,
         0.91066313, -0.33875524],
       [-0.95359166,  1.04243252, -1.12055552, -0.6538297 ,  0.98423908,
         1.91293883, -0.33329657],
       [-0.81612701, -0.8558

In [85]:
df[objectives]

,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,-0.592536,-0.376839,-0.677554,-0.247061,-1.318160,-0.370580,-0.329714
1,0.961022,1.890786,0.585893,1.319077,1.185782,-0.758987,0.986191
2,-0.480038,-0.288479,-0.583600,-0.183466,-1.313598,-0.127347,-0.009533
3,-0.876063,-0.696218,-1.255442,-0.899894,-0.475639,-0.514913,-0.259347
4,-0.746316,-0.619940,0.578864,0.337497,0.333070,3.084435,-0.113739
...,...,...,...,...,...,...,...
4311,-1.196693,-0.315187,-1.305526,-0.822008,-0.010866,-0.650696,-0.371469
4312,-1.563574,0.855251,-1.400834,-0.559283,0.376932,-0.630812,-0.374776
4313,-1.515542,0.745532,-1.412664,-0.521732,0.075399,-0.593704,-0.453170
4314,-1.218929,-0.759734,-1.034383,-0.856490,-0.106618,-0.746913,-0.483224


In [86]:
fronts = fast_non_dominated_sort_indices(data)
pf_front = data[fronts[0]]
pf_data = df.loc[fronts[0]]

In [87]:
from collections import Counter

cats = dict(Counter(pf_data["BREEDGROUP"]))

df[objectives]

,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,-0.592536,-0.376839,-0.677554,-0.247061,-1.318160,-0.370580,-0.329714
1,0.961022,1.890786,0.585893,1.319077,1.185782,-0.758987,0.986191
2,-0.480038,-0.288479,-0.583600,-0.183466,-1.313598,-0.127347,-0.009533
3,-0.876063,-0.696218,-1.255442,-0.899894,-0.475639,-0.514913,-0.259347
4,-0.746316,-0.619940,0.578864,0.337497,0.333070,3.084435,-0.113739
...,...,...,...,...,...,...,...
4311,-1.196693,-0.315187,-1.305526,-0.822008,-0.010866,-0.650696,-0.371469
4312,-1.563574,0.855251,-1.400834,-0.559283,0.376932,-0.630812,-0.374776
4313,-1.515542,0.745532,-1.412664,-0.521732,0.075399,-0.593704,-0.453170
4314,-1.218929,-0.759734,-1.034383,-0.856490,-0.106618,-0.746913,-0.483224


In [88]:
pf_data = pf_data[["BREEDGROUP"] + objectives]
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,Siberian_and_Neva_Masquerade,-0.592536,-0.376839,-0.677554,-0.247061,-1.318160,-0.370580,-0.329714
3,Sphynx_and_Devon_Rex,-0.876063,-0.696218,-1.255442,-0.899894,-0.475639,-0.514913,-0.259347
5,European,0.379262,-0.812642,-0.343483,-0.722727,0.502684,-1.142007,-0.659040
7,Siamese_and_Balinese,-1.008219,-0.673436,-1.421480,-0.341644,-0.850675,0.910663,-0.338755
10,Burmese,-0.813447,0.168712,-1.363108,-0.718478,-0.661025,-0.545538,-0.033225
...,...,...,...,...,...,...,...,...
4309,Landrace_Cat_Shorthair,0.332575,-0.836463,-0.957544,-0.033670,-0.774068,0.033412,-0.416599
4311,Korat,-1.196693,-0.315187,-1.305526,-0.822008,-0.010866,-0.650696,-0.371469
4312,Korat,-1.563574,0.855251,-1.400834,-0.559283,0.376932,-0.630812,-0.374776
4313,Korat,-1.515542,0.745532,-1.412664,-0.521732,0.075399,-0.593704,-0.453170


In [89]:
# normalized_df=(df-df.min())/(df.max()-df.min())

pf_data[objectives] = ((pf_data[objectives] - pf_data[objectives].min()) / (pf_data[objectives].max() - pf_data[objectives].min()) * 100).round(decimals=2)
pf_data[["activity_playfulness", "cat_sociability", "human_sociability"]] = -pf_data[["activity_playfulness", "cat_sociability", "human_sociability"]]

In [90]:
pf_data.round(decimals=2)

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,Siberian_and_Neva_Masquerade,26.30,15.78,-29.70,-33.80,-21.17,20.56,10.14
3,Sphynx_and_Devon_Rex,19.47,9.59,-20.35,-21.88,-32.93,18.00,10.92
5,European,49.71,7.34,-35.11,-25.11,-46.58,6.88,6.50
7,Siamese_and_Balinese,16.28,10.03,-17.66,-32.07,-27.70,43.28,10.04
10,Burmese,20.98,26.35,-18.61,-25.19,-30.34,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,Landrace_Cat_Shorthair,48.58,6.87,-25.17,-37.69,-28.76,27.73,9.18
4311,Korat,11.74,16.97,-19.54,-23.30,-39.41,15.60,9.68
4312,Korat,2.91,39.65,-18.00,-28.10,-44.82,15.95,9.64
4313,Korat,4.06,37.52,-17.81,-28.78,-40.62,16.61,8.78


In [67]:
# pf_data.to_csv("./data/pf_data.csv", index=False)

In [68]:
cats = dict(Counter(pf_data["BREEDGROUP"]))
cats

{'Siberian_and_Neva_Masquerade': 52,
 'Sphynx_and_Devon_Rex': 41,
 'European': 55,
 'Siamese_and_Balinese': 40,
 'Burmese': 28,
 'British': 28,
 'Oriental': 61,
 'Korat': 21,
 'House_cat': 211,
 'Sacred_Birman': 26,
 'Somali': 29,
 'Landrace_Cat_Shorthair': 263,
 'Maine_Coon': 48,
 'Other': 30,
 'Abyssinian': 44,
 'Turkish_Angora': 28,
 'Bengal': 40,
 'American_Curl': 42,
 'Landrace_Cat_Longhair': 50,
 'Norwegian_Forest_Cat': 29,
 'Ocicat': 45,
 'Russian_Blue': 12,
 'Turkish_Van': 10,
 'Ragdoll': 37,
 'Cornish_Rex': 25,
 'Persian_and_Exotic': 15}

In [69]:
cats_as_num = {f"{key}": i for (i, key) in enumerate(cats)}

In [70]:
cats_as_num

{'Siberian_and_Neva_Masquerade': 0,
 'Sphynx_and_Devon_Rex': 1,
 'European': 2,
 'Siamese_and_Balinese': 3,
 'Burmese': 4,
 'British': 5,
 'Oriental': 6,
 'Korat': 7,
 'House_cat': 8,
 'Sacred_Birman': 9,
 'Somali': 10,
 'Landrace_Cat_Shorthair': 11,
 'Maine_Coon': 12,
 'Other': 13,
 'Abyssinian': 14,
 'Turkish_Angora': 15,
 'Bengal': 16,
 'American_Curl': 17,
 'Landrace_Cat_Longhair': 18,
 'Norwegian_Forest_Cat': 19,
 'Ocicat': 20,
 'Russian_Blue': 21,
 'Turkish_Van': 22,
 'Ragdoll': 23,
 'Cornish_Rex': 24,
 'Persian_and_Exotic': 25}

In [71]:
pf_data['BREEDGROUP'] = pf_data['BREEDGROUP'].map(cats_as_num)

In [72]:
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,0,26.30,15.78,29.70,33.80,21.17,20.56,10.14
3,1,19.47,9.59,20.35,21.88,32.93,18.00,10.92
5,2,49.71,7.34,35.11,25.11,46.58,6.88,6.50
7,3,16.28,10.03,17.66,32.07,27.70,43.28,10.04
10,4,20.98,26.35,18.61,25.19,30.34,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,11,48.58,6.87,25.17,37.69,28.76,27.73,9.18
4311,7,11.74,16.97,19.54,23.30,39.41,15.60,9.68
4312,7,2.91,39.65,18.00,28.10,44.82,15.95,9.64
4313,7,4.06,37.52,17.81,28.78,40.62,16.61,8.78


In [73]:
pf_data

,BREEDGROUP,fearfulness,human_aggression,activity_playfulness,cat_sociability,human_sociability,litterbox_issues,excessive_grooming
0,0,26.30,15.78,29.70,33.80,21.17,20.56,10.14
3,1,19.47,9.59,20.35,21.88,32.93,18.00,10.92
5,2,49.71,7.34,35.11,25.11,46.58,6.88,6.50
7,3,16.28,10.03,17.66,32.07,27.70,43.28,10.04
10,4,20.98,26.35,18.61,25.19,30.34,17.46,13.42
...,...,...,...,...,...,...,...,...
4309,11,48.58,6.87,25.17,37.69,28.76,27.73,9.18
4311,7,11.74,16.97,19.54,23.30,39.41,15.60,9.68
4312,7,2.91,39.65,18.00,28.10,44.82,15.95,9.64
4313,7,4.06,37.52,17.81,28.78,40.62,16.61,8.78


In [74]:
conversion = pd.DataFrame([cats_as_num])

In [75]:
conversion

,Siberian_and_Neva_Masquerade,Sphynx_and_Devon_Rex,European,Siamese_and_Balinese,Burmese,British,Oriental,Korat,House_cat,Sacred_Birman,...,Bengal,American_Curl,Landrace_Cat_Longhair,Norwegian_Forest_Cat,Ocicat,Russian_Blue,Turkish_Van,Ragdoll,Cornish_Rex,Persian_and_Exotic
0,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25


In [31]:
conversion.to_csv("./data/conversion.csv", index=False)